In [6]:
import sys
import json
from openai import OpenAI

In [14]:
input_notebook = "auto_markdown.ipynb"
with open(input_notebook, 'r', encoding='utf-8') as f:
    nb_data = json.load(f)

# Clean the data (Remove 'outputs')
for cell in nb_data['cells']:
    if 'outputs' in cell:
        cell['outputs'] = []
    if 'execution_count' in cell:
        cell['execution_count'] = None
        
notebook_json_string = json.dumps(nb_data, indent=2)

In [ ]:
output_notebook = input_notebook.split(".")[0] + "_with_md.ipynb"

In [16]:
ollama_url = "http://localhost:11434/v1"
ollama = OpenAI(api_key="ollama", base_url=ollama_url)
model = "deepseek-coder-v2"

In [3]:
system_prompt = f"""
You are an expert Python Developer and Technical Writer.
Your task is to create meaningful markdown text for the Python code blocks in a Jupyter Notebook
"""

def user_prompt_for(jupyter_nb):
    return f"""
Read this Jupyter Notebook JSON code it to a valid professional-grade  Jupyter Notebook (.ipynb) JSON string. 
Rules:
No Code Changes: Do not modify, refactor, or delete any code within the code cells.
Contextual Markdown: For every code cell in the notebook, insert a new markdown cell immediately preceding it.
Content of Markdown: >    - Add a relevant Header (e.g., ### Data Processing).
Write 2-3 sentences explaining the purpose of the code block.
Describe any inputs, outputs, or side effects (like file saving or plotting).
Preserve Metadata: Keep the existing kernelspec and language_info exactly as they are in the input.
Output Format: Provide the complete, valid JSON for the updated .ipynb file. Output ONLY the JSON.

Input Notebook JSON:
```
{jupyter_nb}
```
"""

In [ ]:
def messages_for(jupyter_nb):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(jupyter_nb)}
    ]

In [12]:
def write_output(json):
    with open(output_notebook, "w") as f:
        f.write(json)

In [17]:
response = ollama.chat.completions.create(model=model, messages=messages_for(notebook_json_string))
reply = response.choices[0].message.content
reply.replace("```json", "").replace("```", "").strip()

write_output(reply)